## STP戦略
</br>
キーワードに付随した,競合を知る</br>
欲：メタ情報から共通項を知る</br>
欲：データフレーム→CSVの掃き出しでもいいかも

In [1]:
#analyticsライブラリ
import os
import re
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import torch
import datetime
import MeCab
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#Google連携ライブラリ
import google.oauth2.credentials
import google_auth_oauthlib.flow
from apiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

C:\Users\kohiz\.conda\envs\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 認証情報

In [2]:
# API情報
#kohizeri = 'hoge'
#youtube_data = 'hoge'
DEVELOPER_KEY = 'hoge'
youtube_api_dic = {'youtube':'v3', 'youtubereporting':'v1'}
YOUTUBE_API_SERVICE_NAME = list(youtube_api_dic.keys())
YOUTUBE_API_VERSION = list(youtube_api_dic.values())

#YouTube_Data_API
youtube_data = build(YOUTUBE_API_SERVICE_NAME[0], 
                     YOUTUBE_API_VERSION[0],
                     developerKey=DEVELOPER_KEY)

#YouTube_Analytics_Reporting_API
def youtube_reporting():
    CLIENT_SECRETS_FILE = 'client_secrets.json'
    ##評価系or収益系のスコープ(デフォは評価を指定)
    SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly',
              'https://www.googleapis.com/auth/yt-analytics-monetary.readonly']
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES[0])
    credentials = flow.run_console()
    return build(YOUTUBE_API_SERVICE_NAME[1], 
                 YOUTUBE_API_VERSION[1],
                 credentials = credentials,
                 developerKey=DEVELOPER_KEY)

## 検索用関数

In [3]:
#他パラメータ
#channelType       :チャンネルタイプ
#eventType         :配信
#publishedAfter    :date(YYYY-MM-DD)
#publishedBefore   :date(YYYY-MM-DD)
#regionCode        :国
#topicId           :トピックス
#videoCategoryId   :カテゴリー

#YouTubeから特定条件で検索する(指定回数分でループ)
def search_youtube(q, order: str, loop_num, maxResults: int, date_after):
    
    nextPageToken = None
    lst = []
    
    #条件検索
    for i in range(0, loop_num):
        result = youtube_data.search()\
                             .list(q=q, 
                                   part='id',
                                   maxResults=maxResults,
                                   order=order, 
                                   pageToken=nextPageToken,
                                   safeSearch='none',
                                   publishedAfter=date_after,
                                   type='video, channel, playlist')\
                             .execute()
        
        #動画IDをリスト化
        for l in range(0, len(result['items'])):
            lst.append(result['items'][l]['id']['videoId'])
        
        nextPageToken = result['nextPageToken']
    
    return result, lst 

In [4]:
#他パラメータ
#channelType       :チャンネルタイプ
#eventType         :配信
#publishedAfter    :date(YYYY-MM-DD)
#publishedBefore   :date(YYYY-MM-DD)
#regionCode        :国
#topicId           :トピックス
#videoCategoryId   :カテゴリー

#YouTubeから特定条件で検索する(限界までループ)
def search_youtube_max(q, order: str, maxResults: int, date_after):
    
    nextPageToken = None
    lst = []
    
    #条件検索
    while True:
        result = youtube_data.search()\
                             .list(q=q, 
                                   part='id',
                                   maxResults=maxResults,
                                   order=order, 
                                   pageToken=nextPageToken,
                                   safeSearch='none',
                                   publishedAfter=date_after,
                                   type='video, channel, playlist')\
                             .execute()
        
        #動画IDをリスト化
        for l in range(0, len(result['items'])):
            lst.append(result['items'][l]['id']['videoId'])
        
        #nextPageToken = result['nextPageToken']
    
    return lst

In [5]:
#YouTube_Videosのメタデータ取得(動画IDを引数とする)
def videos_info(result: list):
    lst = []
    return_elements = ['snippet','contentDetails','statistics','topicDetails']
    
    #result分ループ
    for i, video_id in enumerate(result):
        responce = youtube_data.videos()\
                               .list(part=return_elements, id=video_id)\
                               .execute()
        lst.append(responce['items'][0])
        
    return lst

In [6]:
#辞書からDataFrameを作成
def output_df(lst):

    df1 = pd.DataFrame()
    df2 = pd.DataFrame()

    #要素数分ループ
    for i in range(0, len(lst)):
        for _, key1 in enumerate(lst[i].keys()):
            #df1
            if type(lst[i][key1]) != dict:
                df1.loc[0, key1] = lst[i][key1]
            else:
                for _, key2 in enumerate(lst[i][key1].keys()):
                    df1.loc[0, key2] = str(lst[i][key1][key2])
        #dfの結合
        df2 = pd.concat([df2, df1], axis=0)
        df1 = pd.DataFrame()

    #データフレーム化
    return df2

In [7]:
#csvの出力
def output_csv(key: str, date, df):
    file_lst = []
    date = date[:10]
    for file_name in glob.glob(f'csv/*_youtube_data_{key}_{date}.csv'):
        file_lst.append(file_name)

    if file_lst == []:
        df.to_csv(f'csv/0_youtube_data_{key}_{date}.csv')
    else:
        txt = file_lst[-1][:(file_lst[-1].find('_'))]
        txt = int(txt.replace('csv\\', '')) + 1
        df.to_csv(f'csv/{txt}_youtube_data_{key}_{date}.csv')

## 検索

In [8]:
#quotaの割り当てが圧倒的に足りないので期間を短くしないと動きません
# #検索ワード
# key = str(input())
# date = '2021-01-01T00:00:00Z'

# #限界までループするパターン
# result = search_youtube_max(q=key,
#                             order='viewCount',
#                             date_after=date,
#                             maxResults=50, )

# #videoaのメタデータ
# lst = videos_info(result)

# #データフレーム化
# df = output_df(lst)

# #csv出力
# output_csv(key, date, df)

In [9]:
#検索ワード
key = str(input())
date = '2021-01-01T00:00:00Z'

#loop_num × maxResultsが総出力数
token, result = search_youtube(q=key,
                               order='viewCount',
                               date_after=date,
                               loop_num=8,
                               maxResults=50, )

#videoaのメタデータ
lst = videos_info(result)

#データフレーム化
df = output_df(lst)

#csv出力
output_csv(key, date, df)

 弾き語り
